# Anchor explanations for movie sentiment

In this example, we will explain why a certain sentence is classified by a transformer model as having negative or positive sentiment. The model is trained on negative and positive movie reviews.

In [1]:
import numpy as np
import pandas as pd

import spacy
from alibi.explainers import AnchorText
from alibi.datasets import fetch_movie_sentiment
from alibi.utils.download import spacy_model

import torch

from simpletransformers.model import TransformerModel

/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/clive/anaconda3/envs/seldon-

### Load movie review dataset

The `fetch_movie_sentiment` function returns a `Bunch` object containing the features, the targets and the target names for the dataset.

In [2]:
movies = fetch_movie_sentiment()
movies.keys()

dict_keys(['data', 'target', 'target_names'])

In [3]:
data = movies.data
labels = movies.target
target_names = movies.target_names

### Load spaCy model

English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.

In [4]:
model = 'en_core_web_md'
spacy_model(model=model)
nlp = spacy.load(model)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
✔ Linking successful
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/en_core_web_md
-->
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/spacy/data/en_core_web_md
You can now load the model via spacy.load('en_core_web_md')


### Load pre-trained model

In [5]:
model = TransformerModel('roberta', 'roberta-base', args=({'fp16': False}))

I1023 09:09:34.167677 140296339179264 file_utils.py:296] https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json not found in cache or force_download set to True, downloading to /tmp/tmp7dthfzg0
100%|██████████| 898823/898823 [00:00<00:00, 1694816.79B/s]
I1023 09:09:35.210094 140296339179264 file_utils.py:309] copying /tmp/tmp7dthfzg0 to cache at /home/clive/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
I1023 09:09:35.211631 140296339179264 file_utils.py:313] creating metadata file for /home/clive/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
I1023 09:09:35.212780 140296339179264 file_utils.py:322] removing temp file /tmp/tmp7dthfzg0
I1023 09:09:35.724061 140296339179264 file_utils.py:296] https://s3.amazonaws.com/models.huggingface.co/bert/roberta-b

In [7]:
model.model.load_state_dict(torch.load('outputs/pytorch_moviesentiment_roberta_pytorch_model.bin'))

<All keys matched successfully>

### Initialize anchor text explainer

In [8]:
predict_fn = lambda x: model.predict(x)[1].argmax(axis=1)

In [9]:
explainer = AnchorText(nlp, predict_fn)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  7.12it/s]


### Explain a prediction

In [10]:
class_names = movies.target_names

In [11]:
text = data[4]
print(text)

a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .


Prediction:

In [12]:
pred = class_names[predict_fn([text])[0]]
alternative =  class_names[1 - predict_fn([text])[0]]
print('Prediction: %s' % pred)

100%|██████████| 1/1 [00:00<00:00,  7.99it/s]

100%|██████████| 1/1 [00:00<00:00,  7.46it/s]


Prediction: negative


Explanation:

In [13]:
np.random.seed(0)
explanation = explainer.explain(text, threshold=0.95, use_unk=True)

100%|██████████| 1/1 [00:00<00:00,  6.30it/s]

100%|██████████| 1/1 [00:00<00:00,  7.61it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:10<00:00,  1.42it/s]


  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:09<00:00,  1.48it/s]


  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:09<00:00,  1.48it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  5.79it/s]


use_unk=True means we will perturb examples by replacing words with UNKs. Let us now take a look at the anchor. The word 'exercise' basically guarantees a negative prediction.

In [14]:
print('Anchor: %s' % explanation['names'])
print('Precision: %.2f' % explanation['precision'])
print('\nExamples where anchor applies and model predicts %s:' % pred)
print('\n'.join([x[0] for x in explanation['raw']['examples'][-1]['covered_true']]))
print('\nExamples where anchor applies and model predicts %s:' % alternative)
print('\n'.join([x[0] for x in explanation['raw']['examples'][-1]['covered_false']]))

Anchor: ['vapid']
Precision: 1.00

Examples where anchor applies and model predicts negative:
a UNK UNK but UNK opaque and emotionally vapid exercise in UNK UNK UNK .
UNK visually UNK UNK UNK UNK UNK emotionally vapid exercise in style and mystification .
a visually UNK but narratively UNK and emotionally vapid UNK UNK style UNK UNK .
a UNK flashy UNK UNK UNK UNK UNK vapid UNK UNK style and UNK UNK
a visually UNK UNK narratively opaque and UNK vapid UNK UNK style and UNK .
UNK visually flashy UNK narratively opaque UNK UNK vapid UNK UNK UNK and mystification .
a UNK flashy but UNK opaque UNK UNK vapid UNK UNK style UNK mystification .
a visually UNK but UNK UNK and UNK vapid exercise in style and mystification UNK
UNK UNK flashy but UNK UNK and emotionally vapid UNK UNK style and mystification UNK
UNK UNK flashy but narratively opaque UNK emotionally vapid UNK UNK UNK UNK mystification .

Examples where anchor applies and model predicts positive:



### Changing the perturbation distribution
Let's try this with another perturbation distribution, namely one that replaces words by similar words instead of UNKs.

Explanation:

In [15]:
np.random.seed(0)
explanation = explainer.explain(text, threshold=0.95, use_unk=False, sample_proba=0.5)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  7.05it/s]


100%|██████████| 1/1 [00:00<00:00,  7.04it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:10<00:00,  1.35it/s]


...

100%|██████████| 13/13 [00:10<00:00,  1.37it/s]


  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:14<00:00,  1.09s/it]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  5.51it/s]


The anchor now shows that we need more to guarantee the negative prediction:

In [16]:
print('Anchor: %s' % explanation['names'])
print('Precision: %.2f' % explanation['precision'])
print('\nExamples where anchor applies and model predicts %s:' % pred)
print('\n'.join([x[0] for x in explanation['raw']['examples'][-1]['covered_true']]))
print('\nExamples where anchor applies and model predicts %s:' % alternative)
print('\n'.join([x[0] for x in explanation['raw']['examples'][-1]['covered_false']]))

Anchor: ['vapid']
Precision: 0.99

Examples where anchor applies and model predicts negative:
each visually flashy but physically opaque and tactically vapid exercise before brevity and mystification .
a technically flashy but narratively opaque and terminally vapid strenght in comfort and denial .
a tremendously flashy but narratively acrylic and stylistically vapid handstand in minimalist and uselessness .
this mechanically flashy but truely opaque and militarily vapid exercise in designer and hysteria .
a brilliantly flashy but narratively opaque and emotionally vapid exercise as style and mystification .
both visually skimp but nonetheless opaque and emotionally vapid trampoline among style and wrongness .
a visually flashy but narratively outer and emotionally vapid learner in choice and mystification .
this visually glamorous but distinctly opaque and emotionally vapid exercise with style and mystification .
a perfectly flashy but spiritually opaque and emotionally vapid exercise

We can make the token perturbation distribution sample words that are more similar to the ground truth word via the `top_n` argument. Smaller values (default=100) should result in sentences that are more coherent and thus more in the distribution of natural language which could influence the returned anchor. By setting the `use_probability_proba` to True, the sampling distribution for perturbed tokens is proportional to the similarity score between the possible perturbations and the original word. We can also put more weight on similar words via the `temperature` argument. Lower values of `temperature` increase the sampling weight of more similar words. The following example will perturb tokens in the original sentence with probability equal to `sample_proba`. The sampling distribution for the perturbed tokens is proportional to the similarity score between the ground truth word and each of the `top_n` words.

In [17]:
np.random.seed(0)
explanation = explainer.explain(text, threshold=0.95, use_similarity_proba=True, sample_proba=0.5,
                                use_unk=False, top_n=20, temperature=.2)

100%|██████████| 1/1 [00:00<00:00,  7.15it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  4.95it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  5.41it/s]


...

100%|██████████| 13/13 [00:11<00:00,  1.25it/s]


  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:12<00:00,  1.20it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  5.36it/s]


In [17]:
print('Anchor: %s' % explanation['names'])
print('Precision: %.2f' % explanation['precision'])
print('\nExamples where anchor applies and model predicts %s:' % pred)
print('\n'.join([x[0] for x in explanation['raw']['examples'][-1]['covered_true']]))
print('\nExamples where anchor applies and model predicts %s:' % alternative)
print('\n'.join([x[0] for x in explanation['raw']['examples'][-1]['covered_false']]))

Anchor: ['vapid']
Precision: 1.00

Examples where anchor applies and model predicts negative:
a visually flashy but visually opaque and physically vapid exercise throughout style and paranoia .
another visually gaudy but philosophically opaque and emotionally vapid excercise in style and mystification .
a visually gaudy but narratively opaque and emotionally vapid exercise in retro and mystification .
a graphically snazzy but narratively opaque and spiritually vapid weightloss in style and mystification .
a subtly flashy but narratively transparent and emotionally vapid treadmill in minimalism and mystification .
a visually flashy but graphically translucent and terminally vapid exercise into fashion and mystification .
a visually snazzy but stylistically opaque and emotionally vapid excercise in style and mystification .
a anatomically blocky but narratively opaque and emotionally vapid exercise in practicality and mystification .
a visually flashy but narratively translucent and emot